# Introduction

This competition involves **building a machine learning model to answer multiple-choice questions that were created by a large language model (LLM)**. The dataset consists of questions with **five possible answers (labeled A through E)**.

The task is to **predict the top three most probable answers for each question in the test set**. For each question, there is one answer that is considered the most correct, according to the generating LLM.

The specific files are:

    train.csv: This file contains **200 example questions along with the correct answers**. This data should be used to train your machine learning model.

    test.csv: This file contains the questions for which you must predict the answers. Note that the provided test.csv file is just a placeholder; the actual test data will be provided when your submission is scored. The true test data will have a similar format, but will consist of ~4,000 different questions.

    sample_submission.csv: This file shows the correct format for submitting your predictions.

For each question in test.csv, your model should predict the labels of the top three most probable answers, separated by spaces. These predictions should be stored in a new column called 'prediction'. The order of the labels matters, with the first label being the most likely answer according to your model, the second label being the second most likely, and so on.

Finally, **your predictions should be written to a CSV file for submission**. The submission file should have two columns: 'id' and 'prediction'. The 'id' column should match the 'id' column in test.csv, and the 'prediction' column should contain your model's predictions.

# Load the Train Dataset

First, we need to load the dataset using a library like Pandas. In this case, we have train_df which is a DataFrame that contains our training data.

In [1]:
import pandas as pd
train_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
train_df

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D
...,...,...,...,...,...,...,...,...
195,195,What is the relation between the three moment ...,The three moment theorem expresses the relatio...,The three moment theorem is used to calculate ...,The three moment theorem describes the relatio...,The three moment theorem is used to calculate ...,The three moment theorem is used to derive the...,C
196,196,"What is the throttling process, and why is it ...",The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,The throttling process is a steady adiabatic f...,The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,B
197,197,What happens to excess base metal as a solutio...,"The excess base metal will often solidify, bec...",The excess base metal will often crystallize-o...,"The excess base metal will often dissolve, bec...","The excess base metal will often liquefy, beco...","The excess base metal will often evaporate, be...",B
198,198,"What is the relationship between mass, force, ...",Mass is a property that determines the weight ...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is a property that determines the size of...,D


# Tokenize and Format the Dataset

The LLM understands only numbers, not raw text. Therefore, we need to **tokenize our dataset (convert the text into numbers)** and **format it in the way the LLM expects**. In the case of a multiple-choice question answering task, each sample in our dataset will consist of **a context (the question) and five possible responses (the options)**. We would use **a transformer's tokenizer** for this purpose. For example, **assuming the LLM we are using is based on BERT and tokenizer is a BERT tokenizer**.

**The LLM needs to know the correct answer for each question** in the dataset to learn from it. In our dataset, the correct answer is given in the 'answer' column as a letter (A, B, C, D, or E). We need to **convert these letters into indices (0, 1, 2, 3, or 4)** because the LLM works with numbers.

In [2]:
from transformers import AutoTokenizer
from datasets import Dataset

MODEL_DIR = "/kaggle/input/huggingface-bert/"
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR + "bert-large-uncased")

def encode(row):
    # Format the context and the options.
    prompt = str(row['prompt'])
    options = [str(option) for option in row[['A', 'B', 'C', 'D', 'E']].values.tolist()]
    
    answer_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
    correct_answer_id = answer_mapping[row['answer']]

    encoded_rows = []
    # Tokenize the question and the options, and include the correct answer label.
    for idx, option in enumerate(options):
        text_pair = [prompt, option]
        encoded = tokenizer(text_pair, truncation = True, padding = 'max_length', max_length = 512)
        
        # We set the label to 1 if this is the correct answer, otherwise 0.
        encoded['labels'] = 1 if idx == correct_answer_id else 0
        encoded_rows.append(encoded)

    return encoded_rows

encoded_train = []
for _, row in train_df.iterrows():
    encoded_train.extend(encode(row))

# Now each item in encoded_train is a dictionary representing a single example.
# We can convert it into a Dataset.
encoded_train_dataset = Dataset.from_dict({key: [dic[key] for dic in encoded_train] for key in encoded_train[0]})

In [3]:
encoded_train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

The encoded_train_dataset is an instance of the Dataset class from the Hugging Face datasets library. This dataset contains preprocessed and tokenized training data that we can use to train a machine learning model.

The features field tells what kind of information each example in the dataset includes. Here, it includes:

    input_ids: These are the tokenized inputs to the model. Each token in the input has been mapped to an ID using the vocabulary of the tokenizer.
    
    token_type_ids: These are used by some models (like BERT) to differentiate between different sequences in the input. For example, it can tell the model where the question ends and where the answer options begin.
    
    attention_mask: This is used to tell the model which parts of the input are actual content and which parts are padding (i.e., meaningless tokens added to make all inputs the same length).
    
    labels: These are the correct answers for each question. This is what the model is trying to predict.

The num_rows field tells you that there are 1,000 examples in this dataset.

Please note that the actual content of the dataset is not shown in this overview. we can access the data using indexing, for example "encoded_train_dataset[0]" to get the first example.

In [4]:
#encoded_train_dataset[0]

# See the Labels

In [5]:
print(encoded_train_dataset['labels'][:10])

[0, 0, 0, 1, 0, 1, 0, 0, 0, 0]


In [6]:
answer_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
train_labels = train_df['answer'].map(answer_mapping)

In [7]:
train_labels

0      3
1      0
2      0
3      2
4      3
      ..
195    2
196    1
197    1
198    3
199    2
Name: answer, Length: 200, dtype: int64

# Initialize and Train the Model

We need to initialize the LLM for fine-tuning. We use a version of the LLM that is **specifically designed for multiple-choice tasks**.

**This time we use BERT-large.** The primary difference between the "base" and "large" versions of BERT models lies in **their size, which is reflected in the number of parameters they have, the number of transformer layers (i.e., the "depth" of the network), and the size of these layers (i.e., the "width" of the network)**. This directly impacts the model's capacity to learn from data, its computational requirements, and its performance on different tasks.

Here's a quick comparison:

    BERT-base: BERT-base models are smaller versions, with 12 transformer layers, each with a hidden size of 768, and 12 attention heads. This results in a total of about 110 million parameters.

    BERT-large: BERT-large models are much bigger, with 24 transformer layers, each with a hidden size of 1024, and 16 attention heads. This results in a total of about 340 million parameters.

**Because BERT-large models are larger and have more parameters, they have a greater capacity to learn and model complex patterns in data.** As a result, they typically perform better on tasks involving understanding natural language. **However, they also require more computational resources (both for training and inference), and the improvements they provide may not always justify the increased computational cost**, depending on the specific application and available resources.

**The uncased model does not distinguish between uppercase and lowercase letters (it lowercases all input before tokenizing), whereas the cased model does keep the original letter cases.**

**Training LLMs from scratch is computationally expensive and can take a very long time, even on multiple GPUs**. In practice, **we often use a pre-trained LLM and fine-tune it on our specific task, which is much quicker and requires less computational resources**.

## Compute Metrics

This code **computes the average precision at 3 for each question, then takes the mean of these scores**. The average_precision_at_3 function returns the precision at the rank of the correct label if it is within the top 3 predictions, or 0 otherwise. It uses the index method to find the rank of the correct label, adding 1 because index is 0-based while ranks are 1-based. The try/except block handles the case where the correct label is not in the top 3 predictions.

In [8]:
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    map3 = mean_average_precision_at_3(labels, preds)
    return {
        'map3': map3
    }

def mean_average_precision_at_3(labels, preds):
    ap3s = [average_precision_at_3(label, pred) for label, pred in zip(labels, preds)]
    return np.mean(ap3s)

def average_precision_at_3(label, pred):
    pred_top3 = pred[:3]
    indices = np.where(pred_top3 == label)[0]
    if indices.size > 0:
        # np.where returns 0-based indices, so we add 1 to get the rank.
        return 1 / (indices[0] + 1)
    else:
        return 0

## Train the Model with Cross-Validation

Finally, we can train the model using a library like Hugging Face's Transformers, which provides an easy-to-use Trainer class. We need to provide our encoded dataset, the correct labels, and some training arguments to the Trainer, and then call the train method to start training.

The Hugging Face's Trainer class does not support cross-validation directly. However, we can implement cross-validation yourself by creating multiple splits of your dataset and training a model on each split. **Scikit-Learn's KFold class** can help you create the splits.

In this example, we are using **3-fold cross-validation instead of 5-fold for the limitation on the available GPU**, which means the data is split into 3 subsets (folds). **The model is trained 3 times, each time on 2 of the subsets and evaluated on the remaining subset.**

For the same reason, **the number of epochs and tha batch size are limited to 3 and 1**, respectively.
   
### **Therefore, the learning rate should be the great factor to determine the performance of the model. Please compare with previous versions.**

In [9]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from sklearn.model_selection import KFold

# Disable wandb globally.
import os
os.environ["WANDB_DISABLED"] = "true"

kfold = KFold(n_splits = 3)

for fold, (train_ids, test_ids) in enumerate(kfold.split(encoded_train_dataset)):
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Subset the dataset.
    train_subset = encoded_train_dataset.select(train_ids)
    test_subset = encoded_train_dataset.select(test_ids)
    
    # Create a new instance of the model for each fold.
    model = AutoModelForMultipleChoice.from_pretrained(MODEL_DIR + "bert-large-uncased")

    training_args = TrainingArguments(
        output_dir = f'./finetuned_bert_fold_{fold}',  # output directory for each fold
        num_train_epochs = 3,
        per_device_train_batch_size = 1,
        learning_rate = 2.1e-5,
        gradient_accumulation_steps = 2,
        report_to = [],  # Disable all integrations.
    )

    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = train_subset,
        eval_dataset = test_subset,  # Use the test subset for evaluation.
        compute_metrics = compute_metrics,  # optional function to compute metrics for evaluation
    )

    trainer.train()

    # We can evaluate and save the model after training if we wish.
    eval_result = trainer.evaluate(eval_dataset = test_subset)
    print(f"Eval result: {eval_result}")
    trainer.save_model(f'./finetuned_bert_fold_{fold}')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

FOLD 0
--------------------------------


Some weights of the model checkpoint at /kaggle/input/huggingface-bert/bert-large-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at

Step,Training Loss
500,0.870700


Eval result: {'eval_loss': 0.974872350692749, 'eval_map3': 0.0, 'eval_runtime': 39.8061, 'eval_samples_per_second': 8.391, 'eval_steps_per_second': 1.055, 'epoch': 3.0}
FOLD 1
--------------------------------


...

Step,Training Loss
500,0.848500


Eval result: {'eval_loss': 0.7193840146064758, 'eval_map3': 0.0, 'eval_runtime': 39.6063, 'eval_samples_per_second': 8.408, 'eval_steps_per_second': 1.06, 'epoch': 3.0}
FOLD 2
--------------------------------


Some weights of the model checkpoint at /kaggle/input/huggingface-bert/bert-large-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at

Step,Training Loss
500,0.820400


Eval result: {'eval_loss': 0.9192283153533936, 'eval_map3': 0.0, 'eval_runtime': 39.7091, 'eval_samples_per_second': 8.386, 'eval_steps_per_second': 1.058, 'epoch': 3.0}


# Predict the Test Data

We will make predictions with the trained model and test data.

In [10]:
test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
test_df

,id,prompt,A,B,C,D,E
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...
...,...,...,...,...,...,...,...
195,195,What is the relation between the three moment ...,The three moment theorem expresses the relatio...,The three moment theorem is used to calculate ...,The three moment theorem describes the relatio...,The three moment theorem is used to calculate ...,The three moment theorem is used to derive the...
196,196,"What is the throttling process, and why is it ...",The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,The throttling process is a steady adiabatic f...,The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...
197,197,What happens to excess base metal as a solutio...,"The excess base metal will often solidify, bec...",The excess base metal will often crystallize-o...,"The excess base metal will often dissolve, bec...","The excess base metal will often liquefy, beco...","The excess base metal will often evaporate, be..."
198,198,"What is the relationship between mass, force, ...",Mass is a property that determines the weight ...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is a property that determines the size of...


Encoding: This is the step we are performing with our encode_test() function. Each prompt and option pair is tokenized.

In [11]:
def encode_test(example):
    # Format the context and the options.
    prompt = str(example['prompt'])
    options = [str(option) for option in example[['A', 'B', 'C', 'D', 'E']].values.tolist()]
    examples = []

    # Tokenize the question and the options.
    for option in options:
        text_pair = [prompt, option]
        encoded = tokenizer(text_pair, truncation = True, padding = 'max_length', max_length = 512)
        examples.append(encoded)

    return examples

encoded_test_df = test_df.apply(encode_test, axis = 1)
encoded_test_df

0      [[input_ids, token_type_ids, attention_mask], ...
1      [[input_ids, token_type_ids, attention_mask], ...
2      [[input_ids, token_type_ids, attention_mask], ...
3      [[input_ids, token_type_ids, attention_mask], ...
4      [[input_ids, token_type_ids, attention_mask], ...
                             ...                        
195    [[input_ids, token_type_ids, attention_mask], ...
196    [[input_ids, token_type_ids, attention_mask], ...
197    [[input_ids, token_type_ids, attention_mask], ...
198    [[input_ids, token_type_ids, attention_mask], ...
199    [[input_ids, token_type_ids, attention_mask], ...
Length: 200, dtype: object

Prediction: Next, we need to loop over the encoded inputs, feed them to the model, and store the model outputs.

In [12]:
import torch

# Check if a GPU is available and if not, default to CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Reduce batch size and limit sequence length.
batch_size = 2
sequence_length = 128

predictions = []
for row in encoded_test_df:
    # Truncate or pad sequences to a fixed length.
    row = row[:sequence_length]

    # Create tensors for input_ids and attention_mask.
    input_ids = torch.tensor([item['input_ids'] for item in row], dtype = torch.long).to(device)
    attention_mask = torch.tensor([item['attention_mask'] for item in row], dtype = torch.long).to(device)

    # Run inference with reduced batch size.
    with torch.no_grad():
        outputs = model(input_ids = input_ids, attention_mask = attention_mask)

    predictions.append(outputs.logits.detach().cpu().numpy())

    # Free GPU memory by deleting tensors.
    del input_ids, attention_mask, outputs

In [13]:
predictions[0:5]

[array([[ 1.586057 , -3.0604074],
        [ 1.586057 , -2.964359 ],
        [ 1.586057 , -3.067085 ],
        [ 1.586057 , -2.9638898],
        [ 1.586057 , -3.0421693]], dtype=float32),
 array([[ 1.5568192, -2.9404914],
        [ 1.5568192, -2.9486485],
        [ 1.5568192, -2.9299943],
        [ 1.5568192, -2.946149 ],
        [ 1.5568192, -2.9360266]], dtype=float32),
 array([[ 1.6091564, -2.8757195],
        [ 1.6091564, -3.029059 ],
        [ 1.6091564, -3.003296 ],
        [ 1.6091564, -3.0344126],
        [ 1.6091564, -3.077248 ]], dtype=float32),
 array([[ 1.6203096, -2.9147065],
        [ 1.6203096, -2.9702091],
        [ 1.6203096, -2.944651 ],
        [ 1.6203096, -2.9809136],
        [ 1.6203096, -2.9902933]], dtype=float32),
 array([[ 1.5371877, -2.7604625],
        [ 1.5371877, -2.7619119],
        [ 1.5371877, -2.756531 ],
        [ 1.5371877, -2.7581959],
        [ 1.5371877, -2.7513578]], dtype=float32)]

# Submission

In [14]:
import numpy as np

# Convert the list of predictions to a numpy array.
predictions = np.array(predictions)

# Get the indices of the top 3 predictions for each question.
top_three_indices = (-predictions).argsort(axis = 1)[:, :3].tolist()

In [15]:
# Initialize an empty list to store the extracted values.
top_values = []

# Loop over all elements in the 'top_three_indices' list.
for i in range(len(top_three_indices)):
    # Use a list comprehension to extract the second element (index 1) from each sublist.
    # This will create a new list 'values' containing these three elements.
    values = [top_three_indices[i][j][1] for j in range(3)]
    # Append this new list to our 'top_values' list.
    top_values.append(values)

# Print the resulting list of lists.
print(top_values)

[[3, 1, 4], [2, 4, 0], [0, 2, 1], [0, 2, 1], [4, 2, 3], [1, 2, 4], [4, 0, 1], [3, 0, 1], [2, 3, 1], [0, 2, 4], [2, 1, 0], [0, 3, 1], [2, 1, 4], [4, 1, 3], [1, 2, 3], [1, 0, 2], [3, 1, 4], [4, 0, 3], [0, 3, 1], [4, 2, 0], [3, 1, 4], [3, 2, 4], [2, 0, 4], [2, 1, 4], [4, 0, 3], [4, 0, 3], [0, 3, 2], [3, 1, 2], [4, 1, 0], [2, 1, 3], [1, 2, 0], [4, 2, 3], [4, 3, 1], [1, 3, 4], [4, 3, 0], [1, 3, 4], [1, 2, 0], [0, 3, 4], [4, 3, 0], [2, 0, 4], [4, 1, 0], [4, 1, 0], [1, 4, 3], [2, 0, 4], [0, 3, 1], [0, 1, 3], [1, 4, 2], [2, 3, 0], [3, 2, 0], [1, 4, 3], [1, 3, 0], [4, 3, 2], [2, 0, 4], [0, 2, 4], [1, 0, 3], [0, 3, 2], [2, 3, 1], [2, 1, 0], [0, 3, 4], [1, 3, 2], [1, 2, 0], [3, 4, 1], [2, 0, 3], [2, 1, 4], [3, 0, 1], [4, 3, 0], [2, 0, 3], [4, 3, 1], [2, 4, 3], [3, 0, 4], [0, 2, 1], [0, 4, 1], [3, 0, 4], [1, 2, 3], [3, 0, 4], [0, 1, 3], [3, 2, 0], [1, 4, 2], [2, 4, 3], [4, 0, 2], [3, 2, 4], [0, 4, 2], [4, 2, 1], [0, 3, 2], [2, 4, 0], [2, 3, 1], [3, 4, 2], [4, 1, 2], [2, 4, 0], [3, 1, 0], [3, 1, 0]

In [16]:
# Define a mapping from indices to labels.
index_to_label = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}

# Convert the top three indices to the required format (labels separated by spaces).
top_three_labels = [' '.join([index_to_label[idx] for idx in sublist]) for sublist in top_values]
top_three_labels[0:5]

['D B E', 'C E A', 'A C B', 'A C B', 'E C D']

In [17]:
# Create a new DataFrame for the submission.
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'prediction': top_three_labels
})

# Save the submission DataFrame to a .csv file.
submission_df.to_csv('submission.csv', index = False)

In [18]:
submission_df

,id,prediction
0,0,D B E
1,1,C E A
2,2,A C B
3,3,A C B
4,4,E C D
...,...,...
195,195,E C A
196,196,B E C
197,197,E B D
198,198,C B D


# Conclusion

**You will understand the basic concept as to the use of LLMs with the data.**

I am a medical doctor working on **artificial intelligence (AI) for medicine**. At present AI is also widely used in the medical field. Particularly, AI performs in the healthcare sector following tasks: **image classification, object detection, semantic segmentation, GANs, text classification, etc**. **If you are interested in AI for medicine, please see my other notebooks.**